This notebook may possible charge you lots of money.
Use with care.

In [44]:
import googlemaps
from datetime import datetime, timedelta
from pprint import pprint
import pytz
import os
import json
import gmaps as gmv
import numpy as np
import itertools
import sys

Load configuration

In [45]:
base_path = os.path.join('C:\\', 'Users', 'glenn', 'src', 'pycommute')
config_file = os.path.join(base_path, 'config.json')
with open(config_file, 'r') as f:
    config = json.load(f)

In [46]:
local_tz = pytz.timezone(config['time_zone'])
arrival_time = datetime.now(local_tz).replace(hour=8, minute=0, second=0, microsecond=0) + timedelta(days=1) # Tomorrow at 8
while arrival_time.weekday() != 0: # Finding the next Monday.
    arrival_time += timedelta(days=1)
    
print(f'Calculating for arrival at {arrival_time}.')

Calculating for arrival at 2020-04-13 08:00:00+02:00.


Initialize API

In [47]:
gmd = googlemaps.Client(key=config['api_key']) # Data API

Define function to calculate a point from a origin and distance and heading.

In [48]:
def point_from_distance(lat0d, lon0d, distance, heading):
    if heading == 'north':
        psi = 0
    elif heading == 'east':
        psi = -np.pi/2
    elif heading == 'south':
        psi = np.pi
    elif heading == 'west':
        psi = np.pi/2
    else:
        psi = np.radians(heading)
    
    G = 6371000 # great circle radius
    d = distance / G
    
    lat0 = np.radians(lat0d)
    lon0 = np.radians(lon0d)
    
    lat = np.arcsin(np.sin(lat0) * np.cos(d) + np.cos(lat0) * np.sin(d) * np.cos(psi))
    if abs(np.cos(lat)) <= 0.001:
        lon = lon0
    else:
        lon = ((lon0 - np.arcsin(np.sin(psi) * np.sin(d) / np.cos(lat)) + np.pi) % (2*np.pi)) - np.pi
    
    return np.degrees(lat), np.degrees(lon)

Create grid

In [49]:
lat_center = (config['northeast'][0]+config['southwest'][0])/2
lon_center = (config['northeast'][1]+config['southwest'][1])/2
dlat = point_from_distance(lat_center, lon_center, config['resolution'], 'north')[0] - lat_center
dlon = point_from_distance(lat_center, lon_center, config['resolution'], 'east')[1] - lon_center
latitudes = np.arange(config['southwest'][0], config['northeast'][0], dlat)
longitudes = np.arange(config['southwest'][1], config['northeast'][1], dlon)
origins = list(itertools.product(latitudes, longitudes))
destinations_geocodes = [gmd.geocode(destination)[0] for destination in config['destinations']]
destinations = [(gc['geometry']['location']['lat'], gc['geometry']['location']['lng']) for gc in destinations_geocodes]
print(f'Number of origins: {len(origins)}, number of destinations: {len(config["destinations"])}. {len(config["destinations"]) * len(origins) * 2} elements is approximately {len(config["destinations"]) * len(origins) * 2 * 0.005} USD.')

Number of origins: 204, number of destinations: 2. 816 elements is approximately 4.08 USD.


Since the distance_matrix api can only handle requests with maximum 25 origins and destinations, we split the origins into batches of 25.

In [50]:
max_distance_matrix_locations = 25
i_origins = iter(origins)
origins_batches = []
while True:
    next_batch = list(itertools.islice(i_origins, max_distance_matrix_locations))
    if not next_batch:
        break
    origins_batches.append(next_batch)

Options that stay the same

In [51]:
static_args = {
    'language': config.get('language', 'english'),
    'units': config.get('units', 'metric'),
    'destinations': destinations,
    'arrival_time': arrival_time,
}

Calculate distance with transit

In [52]:
err_log_transit = []
distance_matrices_transit = []
batches_transit = []
for batch in origins_batches:
    try:
        distance_matrix = gmd.distance_matrix(origins=batch, mode='transit', **static_args)
        distance_matrices_transit.append(distance_matrix)
        batches_transit.append(batch)
    except:
        ex = sys.exc_info()[0]
        print(repr(ex))
        err_log_transit.append(ex)
        if len(err_log_transit) >= 10:
            break


In [1]:
# distance_matrices_transit[0]

Calculate distances with driving

In [54]:

err_log_driving = []
distance_matrices_driving = []
batches_driving = []
for batch in origins_batches:
    # try:
    distance_matrix = gmd.distance_matrix(origins=batch, mode='driving', **static_args)
    distance_matrices_driving.append(distance_matrix)
    batches_driving.append(batch)
#     except:
#         ex = sys.exc_info()[0]
#         print(ex)
#         err_log_driving.append(ex)
#         if len(err_log_driving) >= 10:
#             break


Concatenate arrays

In [55]:
i_distance_matrices_transit = iter(distance_matrices_transit)
i_distance_matrices_driving = iter(distance_matrices_driving)

distance_matrix_transit = next(i_distance_matrices_transit)
for matrix in i_distance_matrices_transit:
    distance_matrix_transit['origin_addresses'].extend(matrix['origin_addresses'])
    distance_matrix_transit['rows'].extend(matrix['rows'])
    assert len(matrix['origin_addresses']) == len(matrix['rows']), 'Unequal lengths!'

distance_matrix_driving = next(i_distance_matrices_driving)
for matrix in i_distance_matrices_driving:
    distance_matrix_transit['origin_addresses'].extend(matrix['origin_addresses'])
    distance_matrix_transit['rows'].extend(matrix['rows'])
    assert len(matrix['origin_addresses']) == len(matrix['rows']), 'Unequal lengths!'

Store data

In [57]:
data_path = os.path.join(base_path, 'test_data.json')
data_object = {
    'distance_matrix_transit': distance_matrix_transit,
    'distance_matrix_driving': distance_matrix_driving,
    'origins': origins,
    'origins_batches': origins_batches,
    'destinations_geocodes': destinations_geocodes,
    'resolution': config['resolution'],
}

with open(data_path, 'w') as f:
    json.dump(data_object, f, indent=4)
